<img style="float: left;;" src='Figures/alinco.png' /></a>

# Modulo II: Parts-of-Speech Tagging (POS)

Aquí desarrollaremos habilidades en el etiquetado de parte del discurso (POS), el proceso de asignar una etiqueta de parte del discurso (sustantivo, verbo, adjetivo ...) a cada palabra en un texto de entrada. Etiquetar es difícil porque algunas palabras pueden representar más de una parte del discurso en diferentes momentos. Son **Ambiguos**. Veamos el siguiente ejemplo:

- The whole team played **well**. [adverb]
- You are doing **well** for yourself. [adjective]
- **Well**, this assignment took me forever to complete. [interjection]
- The **well** is dry. [noun]
- Tears were beginning to **well** in her eyes. [verb]

Distinguir las partes del discurso de una palabra en una oración nos ayudará a comprender mejor el significado de una oración. Esto sería de vital importancia en las consultas de búsqueda. Identificar el nombre propio, la organización, el símbolo bursátil o algo similar mejoraría enormemente todo, desde el reconocimiento de voz hasta la búsqueda. Veremos:

- Descubrir cómo funciona el etiquetado de partes gramaticales
- Calcular la matriz de transición A en un modelo de Markov oculto
- Calcular la matriz de transición B en un modelo de Markov oculto
- Calcular el algoritmo de Viterbi
- Calcular la precisión del modelo


In [1]:
# Importando las librerías


## Fuente de Datos
Uilizaremos conjuntos de datos etiquetados recopilados del **Wall Street Journal (WSJ)**.

[aquí](http://relearn.be/2015/training-common-sense/sources/software/pattern-2.6-critical-fork/docs/html/mbsp-tags.html) es un ejemplo de "conjunto de etiquetas" o designación de Parte del discurso que describe la etiqueta de dos o tres letras y su significado.

- Se utilizará un conjunto de datos (**WSJ-2_21.pos**) para *entrenamiento**.
- El otro (**WSJ-24.pos**) para **pruebas**.
- Los datos de entrenamiento etiquetados se han procesado previamente para formar un vocabulario (**hmm_vocab.txt**).
- El vocabulario se amplía con un conjunto de "símbolos de palabras desconocidas", que se describen a continuación.

El conjunto de entrenamiento se utilizará para crear los recuentos de emisiones, transmisiones y etiquetas. 

Se lee el conjunto de prueba (WSJ-24.pos) para crear `y`.
- Contiene tanto el texto de prueba como la etiqueta verdadera.
- El conjunto de prueba también se ha procesado previamente para eliminar las etiquetas y formar **test_words.txt**.
- Esto se lee y se procesa para identificar el final de las oraciones y manejar palabras que no están en el vocabulario.
- Esto forma la lista `prep`, el texto preprocesado que se usa para probar nuestros etiquetadores POS.

Un etiquetador de POS encontrará necesariamente palabras que no están en sus conjuntos de datos.

- Para mejorar la precisión, estas palabras se analizan más a fondo durante el preprocesamiento para extraer pistas disponibles en cuanto a su etiqueta apropiada.
- Por ejemplo, el sufijo 'ize' es una pista de que la palabra es un verbo, como en 'final-ize' o 'character-ize'.
- Un conjunto de tokens desconocidos, como '--unk-verb--' o '--unk-noun--' reemplazará las palabras desconocidas tanto en el cuerpo de entrenamiento como en el de prueba y aparecerá en la emisión, transmisión y estructuras de datos de etiquetas.


<img src = "Figures/DataSources1.PNG" />


# Parts-of-speech tagging 


## Entrenamiento
Comenzaremos con el etiquetador de partes del discurso más simple posible.

En esta sección, encontraremos las palabras que no son ambiguas.
- Por ejemplo, la palabra `is` es un verbo y no es ambigua.
- En el corpus `WSJ`, $86$% del token son inequívocos (lo que significa que solo tienen una etiqueta)
- Aproximadamente $14\% $ son ambiguos (lo que significa que tienen más de una etiqueta)


<img src = "Figures/pos.png" style="width:400px;height:250px;"/>

Antes de comenzar a predecir las etiquetas de cada palabra, necesitaremos calcular algunos diccionarios que le ayudarán a generar las tablas.


#### La conteo de las transiciones

- El primer diccionario es el diccionario `transition_counts` que calcula el número de veces que cada etiqueta pasó junto a otra etiqueta.

Este diccionario se utilizará para calcular:
$$P(t_i | t_ {i-1})\tag{1}$$

Ésta es la probabilidad de una etiqueta en la posición $ i $ dada la etiqueta en la posición $ i-1 $.

Para que pueda calcular la ecuación 1, crearemos un diccionario `transiciones_contos` donde
- Las claves son `(prev_tag, tag)`
- Los valores son el número de veces que aparecieron esas dos etiquetas en ese orden.


#### Conteo de Emisiones

El segundo diccionario que se calculará es el diccionario `Emision_counts`. Este diccionario se utilizará para calcular:

$$ P (w_i | t_i) \tag {2} $$

En otras palabras, lo usaremos para calcular la probabilidad de una palabra dada su etiqueta.

Para que se pueda calcular la ecuación 2, crearemos un diccionario de "recuentos_de_emisiones" donde
- Las claves son `(etiqueta, palabra)`
- Los valores son el número de veces que ese par apareció en tu conjunto de entrenamiento.


#### Conteo de etiquetas

El último diccionario que se calculará es el diccionario `tag_counts`.
- La clave es la etiqueta
- El valor es el número de veces que apareció cada etiqueta.


**Instructions:** Escribir una función `training_corpus` que retorne los tres diccionarios mencionado anteriormente `transition_counts`, `emission_counts`, y `tag_counts`. 

- `emision_counts`: asigna (tag, word) al número de veces que sucedió.
- `transition_counts`: asigna (prev_tag, tag) al número de veces que ha aparecido.
- `tag_counts`: asigna (tag) al número de veces que ha ocurrido.


Los 'estados' son las designaciones de las partes del discurso que se encuentran en los datos de entrenamiento. También se denominarán "etiquetas" o POS.

- "NN" es sustantivo, singular,
- 'NNS' es sustantivo, plural.
- Además, hay etiquetas útiles como '--s--' que indican el comienzo de una oración.


### Probando
Ahora probará la precisión del modelo usando el diccionario `missions_counts`.
- Dado el corpus de prueba preprocesado `prep`, asignaremos una etiqueta de POS a cada palabra en ese corpus.
- Usando el corpus de prueba etiquetado original `y`, se calculará qué porcentaje de las etiquetas se obtuvo correctamente.


**Instructions:** Implementar la función `predict_pos` que calcule el accuracy del modelo. 



# Modelo Oculto de Markov para POS

Implementaremos un modelo oculto de Markov (HMM) con el algoritmo  de Viterbi
- El HMM es uno de los algoritmos más utilizados en el procesamiento del lenguaje natural y es la base de muchas técnicas de aprendizaje profundo.
- Además del etiquetado de partes de voz, HMM se utiliza en reconocimiento de voz, síntesis de voz, etc.

El modelo de Markov contiene varios estados y la probabilidad de transición entre esos estados.
- En este caso, los estados son las partes del discurso.
- Un modelo de Markov utiliza una matriz de transición, "A".
- Un modelo de Markov oculto agrega una matriz de observación o emisión "B" que describe la probabilidad de una observación visible cuando estamos en un estado particular.
- En este caso, las emisiones son las palabras del corpus
- El estado, que está oculto, es la etiqueta POS de esa palabra.




## Generación de Matrices

### Creando la matriz 'A' para las probabilidades de las transiciones

Ahora que tenemos los diccionarios `recuentos_emisiones`,` recuentos_transición` y `recuentos_etiquetas`, comenzaremos a implementar el modelo oculto de Markov.

Esto nos permitirá construir rápidamente:
- Matriz de probabilidades de transición `A`.
- y la matriz de probabilidades de emisión `B`.

También utilizaremos el smoothing al calcular estas matrices.

A continuación, se muestra un ejemplo de cómo se vería la matriz de transición `A` (se simplifica a 5 etiquetas para su visualización. Tiene un tamaño de 46x46):

|**A**  |...|         RBS  |          RP  |         SYM  |      TO  |          UH|...
| --- ||---:-------------| ------------ | ------------ | -------- | ---------- |----
|**RBS**  |...|2.217069e-06  |2.217069e-06  |2.217069e-06  |0.008870  |2.217069e-06|...
|**RP**   |...|3.756509e-07  |7.516775e-04  |3.756509e-07  |0.051089  |3.756509e-07|...
|**SYM**  |...|1.722772e-05  |1.722772e-05  |1.722772e-05  |0.000017  |1.722772e-05|...
|**TO**   |...|4.477336e-05  |4.472863e-08  |4.472863e-08  |0.000090  |4.477336e-05|...
|**UH**  |...|1.030439e-05  |1.030439e-05  |1.030439e-05  |0.061837  |3.092348e-02|...
| ... |...| ...          | ...          | ...          | ...      | ...        | ...

Tenga en cuenta que la matriz anterior se calculó aplicando smoothing.

Cada celda le da la probabilidad de pasar de una parte del discurso a otra.
- En otras palabras, hay una probabilidad de 4.47e-8 de pasar de las partes del discurso `TO` a` RP`.
- La suma de cada fila debe ser igual a 1, porque asumimos que la siguiente etiqueta POS debe ser una de las columnas disponibles en la tabla.

El suavizado se realizó de la siguiente manera:

$$ P(t_i | t_{i-1}) = \frac{C(t_{i-1}, t_{i}) + \alpha }{C(t_{i-1}) +\alpha * N}\tag{3}$$

- $ N $ es el número total de etiquetas
- $ C (t_{i-1}, t_{i}) $ es el recuento de la tupla (POS anterior, POS actual) en el diccionario `transition_counts`.
- $ C (t_{i-1}) $ es el recuento del POS anterior en el diccionario `tag_counts`.
- $ \alpha $ es un parámetro de suavizado.



**Instructions:** Implementar la `create_transition_matrix` a continuación para todas las etiquetas. Su tarea es generar una matriz que calcule la ecuación 3 para cada celda en la matriz `A`.


### Crear la matriz de probabilidades de emision 'B' 

Utilizaremos el smoothing como se define a continuación:

$$P(w_i | t_i) = \frac{C(t_i, word_i)+ \alpha}{C(t_{i}) +\alpha * N}\tag{4}$$

- $ C (t_i, word_i) $ es el número de veces que $ word_i $ se asoció con $ tag_i $ en los datos de entrenamiento (almacenados en el diccionario `missions_counts`).
- $ C (t_i) $ es el número de veces que $ tag_i $ estuvo en los datos de entrenamiento (almacenados en el diccionario `tag_counts`).
- $ N $ es el número de palabras del vocabulario
- $ \alpha $ es un parámetro de suavizado.

La matriz `B` es de dimensión (num_tags, N), donde num_tags es el número de posibles etiquetas de partes del discurso.

Aquí hay un ejemplo de la matriz, solo se muestra un subconjunto de etiquetas y palabras:
 
<p style='text-align: center;'> <b>B Matriz de probabilidad de emisiones (subconjunto)</b>  </p>

|**B**| ...|          725 |     adroitly |    engineers |     promoted |      synergy| ...|
|----|----|--------------|--------------|--------------|--------------|-------------|----|
|**CD**  | ...| **8.201296e-05** | 2.732854e-08 | 2.732854e-08 | 2.732854e-08 | 2.732854e-08| ...|
|**NN**  | ...| 7.521128e-09 | 7.521128e-09 | 7.521128e-09 | 7.521128e-09 | **2.257091e-05**| ...|
|**NNS** | ...| 1.670013e-08 | 1.670013e-08 |**4.676203e-04** | 1.670013e-08 | 1.670013e-08| ...|
|**VB**  | ...| 3.779036e-08 | 3.779036e-08 | 3.779036e-08 | 3.779036e-08 | 3.779036e-08| ...|
|**RB**  | ...| 3.226454e-08 | **6.456135e-05** | 3.226454e-08 | 3.226454e-08 | 3.226454e-08| ...|
|**RP**  | ...| 3.723317e-07 | 3.723317e-07 | 3.723317e-07 | **3.723317e-07** | 3.723317e-07| ...|
| ...    | ...|     ...      |     ...      |     ...      |     ...      |     ...      | ...|




**Instructions:** Implementar la función `create_emission_matrix`(` B`). La función tomará $ \alpha $, el parámetro de suavizado, `tag_counts`, que es un diccionario que asigna cada etiqueta a su conteo respectivo, el diccionario `missions_counts` donde están las claves (etiqueta, palabra) y los valores son los conteos. La tarea es crear una función para generar una matriz que calcule la ecuación 4 para cada celda en la matriz `B`.


# Algoritmo de Viterbi y Programación Dinámica

En esta parte, implementaremos el algoritmo de Viterbi que hace uso de la programación dinámica. Específicamente, usaremos las dos matrices, `A` y` B` para calcular el algoritmo de Viterbi. Descompondremos esto en tres paso principales:

* **Inicialización** - En esta parte, se inicializarán las matrices `best_paths` y` best_probabilities` que se completará en `feed_forward`.
* **Feed forward**: en cada paso, calcular la probabilidad de que ocurra cada ruta y las mejores rutas hasta ese punto.
* **Feed backward**: Para encontrar el mejor camino con las mayores probabilidades.


## Inicialización

Comenzaremos inicializando dos matrices de la misma dimensión.

- best_probs: Cada celda contiene la probabilidad de pasar de una etiqueta POS a una palabra en el corpus.

- best_paths: una matriz que le ayuda a seguir la mejor ruta posible en el corpus.


**Instruciones**: 
Escribir un programa que inicialice la matriz `best_probs` y` best_paths`.

Ambas matrices se inicializarán a cero excepto por la columna cero de `best_probs`.
- La columna cero de `best_probs` se inicializa asumiendo que la primera palabra del corpus fue precedida por un token de inicio (" --s-- ").
- Esto permite hacer referencia a la matriz **A** para la probabilidad de transición

A continuación se explica cómo inicializar la columna 0 de `best_probs`:
- La probabilidad de que la mejor ruta vaya desde el índice de inicio hasta una etiqueta POS determinada indexada por el entero $ i $ se indica con $ \textrm {best_probs} [s_{idx}, i] $.
- Esto se estima como la probabilidad de que la etiqueta de inicio pase al POS denotado por el índice $ i $: $ \mathbf {A} [s_{idx}, i] $ Y que la etiqueta POS denotada por $ i $ emita la primera palabra del corpus dado, que es $ \mathbf {B} [i, vocab [corpus [0]]] $.
- Tenga en cuenta que vocab [corpus [0]] se refiere a la primera palabra del corpus (la palabra en la posición 0 del corpus).
- **vocab** es un diccionario que devuelve el número entero único que se refiere a esa palabra en particular.

Conceptualmente, se ve así:

$\textrm{best_probs}[s_{idx}, i] = \mathbf{A}[s_{idx}, i] \times \mathbf{B}[i, corpus[0] ]$

Para evitar multiplicar y almacenar pequeños valores, tomaremos el logaritmo del producto, que se convierte en la suma de dos logaritmos:

$best\_probs[i,0] = log(A[s_{idx}, i]) + log(B[i, vocab[corpus[0]]$

Además, para evitar tomar el logaritmo de 0 (que se define como infinito negativo), el código en sí solo establecerá $ best\_probs [i, 0] = float ('- inf') $ cuando $ A [s_ {idx} , i] == 0 $

Entonces, la implementación para inicializar $ best \_probs $ se ve así:

$ if A[s_{idx}, i] <> 0 : best\_probs[i,0] = log(A[s_{idx}, i]) + log(B[i, vocab[corpus[0]]])$

$ if A[s_{idx}, i] == 0 : best\_probs[i,0] = float('-inf')$



El siguiente ejemplo muestra la inicialización asumiendo que el corpus comienza con la frase "Loss tracks upward".
<img src = "Figures/Initialize4.PNG"/>

Representa el infinito y el infinito negativo de esta manera:

```CPP
float('inf')
float('-inf')
```


## Viterbi Forward

En esta parte, implementaremos la función `viterbi_forward`. En otras palabras, rellenaremos las matrices `best_probs` y` best_paths`.
- buscando hacia adelante a través del corpus.
- Para cada palabra, calcular una probabilidad para cada posible etiqueta.
- A diferencia del algoritmo anterior `predict_pos`, este incluirá la ruta hasta la combinación (palabra, etiqueta)

A continuación, se muestra un ejemplo con un corpus de tres palabras "Loss tracks upward":

- En el diagrama siguiente, la primera palabra "Pérdida" ya está inicializada.
- El algoritmo calculará una probabilidad para cada una de las etiquetas potenciales en la segunda y las futuras palabras.

Calcular la probabilidad de que la etiqueta de la segunda obra ('tracks') sea un verbo, presente en tercera persona del singular (VBZ).

- En la matriz `best_probs`, nos vamos a la columna de la segunda palabra ('tracks') y la fila 40 (VBZ), esta celda está resaltada en naranja en el diagrama que se muestra a continuación.
- Examine cada una de las rutas de las etiquetas de la primera palabra ('loss') y elija la ruta más probable.
- Un ejemplo del cálculo para **una** de esas rutas es la ruta desde ('loss', NN) a ('tracks', VBZ).
- El logaritmo de la probabilidad de la ruta hasta la primera palabra 'loss' incluida que tiene la etiqueta POS NN es $ -14,32 $. La matriz `best_probs` contiene este valor -14,32 en la columna de 'loss' y la fila de 'NN'.
- Encuentre la probabilidad de que NN cambie a VBZ. Para encontrar esta probabilidad, vaya a la matriz de transición "A" y vaya a la fila de "NN" y a la columna de "VBZ". El valor es $ 4.37e-02 $, que está encerrado en un círculo en el diagrama, así que agregue $ -14.32 + log (4.37e-02) $.
- Encuentre el logaritmo de la probabilidad de que la etiqueta VBS 'emita' la palabra 'loss'. Para encontrar esto, mire la matriz de emisión 'B' en la fila 'VBZ' y la columna de la palabra 'tracks'. El valor $ 4.61e-04 $ está encerrado en un círculo en el siguiente diagrama. Entonces agregue $ -14.32 + log (4.37e-02) + log (4.61e-04) $.
- La suma de $ -14.32 + log (4.37e-02) + log (4.61e-04) $ es $ -25.13 $. Almacenar $ -25.13 $ en la matriz `best_probs` en la fila 'VBZ' y la columna 'tracks' (como se ve en la celda que está resaltada en naranja en el diagrama).
- Se calculan todas las demás rutas en best_probs. Observe que $ -25.13 $ es mayor que todos los demás valores en la columna 'tracks' de la matriz `best_probs`, por lo que la ruta más probable a'VBZ' es desde 'NN'. 'NN' está en la fila 20 de la matriz `best_probs`, por lo que $ 20 $ es la ruta más probable.
- Almacene la ruta más probable $ 20 $ en la tabla `best_paths`. Esto se resalta en naranja en el diagrama a continuación.



La fórmula para calcular la probabilidad y la ruta para la palabra $ i^{th} $ en el $ corpus $, la palabra anterior $ i-1 $ en el corpus, la etiqueta POS actual $ j $ y la etiqueta POS anterior $ k $ es:

$\mathrm{prob} = \mathbf{best\_prob}_{k, i-1} + \mathrm{log}(\mathbf{A}_{k, j}) + \mathrm{log}(\mathbf{B}_{j, vocab(corpus_{i})})$

donde $ corpus_{i} $ es la palabra en el corpus en el índice $ i $, y $ vocab $ es el diccionario que obtiene el entero único que representa una palabra dada.

$\mathrm{path} = k$

donde $ k $ es el número entero que representa la etiqueta POS anterior.

**Instrucciones:** Implementar el algoritmo `viterbi_forward` y almacenar el best_path y best_prob para cada etiqueta posible para cada palabra en las matrices` best_probs` y `best_tags` usando el pseudocódigo a continuación.


`for each word in the corpus

    for each POS tag type that this word may be
    
        for POS tag type that the previous word could be
        
            compute the probability that the previous word had a given POS tag, that the current word has a given POS tag, and that the POS tag would emit this current word.
            
            retain the highest probability computed for the current word
            
            set best_probs to this highest probability
            
            set best_paths to the index 'k', representing the POS tag of the previous word which produced the highest probability`


<img src = "Figures/Forward4.PNG"/>


## Viterbi backward
Ahora implementaremos el algoritmo hacia atrás de Viterbi.
- El algoritmo hacia atrás de Viterbi obtiene las predicciones de las etiquetas POS para cada palabra en el corpus utilizando las matrices `best_paths` y` best_probs`.

El siguiente ejemplo muestra cómo caminar hacia atrás a través de la matriz best_paths para obtener las etiquetas POS de cada palabra en el corpus. Recuerde que este corpus de ejemplo tiene tres palabras: "Loss tracks upward".

La etiqueta POS para "upward" es "RB"
- Seleccione la etiqueta POS más probable para la última palabra del corpus, 'upward' en la tabla `best_prob`.
- Busque la fila en la columna de 'hacia arriba' que tiene la mayor probabilidad.
- Observe que en la fila 28 de `best_probs`, la probabilidad estimada es -34,99, que es mayor que los otros valores de la columna. Entonces, la etiqueta POS más probable para 'upward' es `RB` un adverbio, en la fila 28 de` best_prob`.
- La variable `z` es una matriz que almacena el ID entero único de las etiquetas POS predichas para cada palabra en el corpus. En la matriz z, en la posición 2, almacene el valor 28 para indicar que la palabra 'hacia arriba' (en el índice 2 en el corpus), muy probablemente tiene la etiqueta POS asociada con la identificación única 28 (que es 'RB').
- La variable `pred` contiene las etiquetas POS en forma de cadena. Entonces, `pred` en el índice 2 almacena la cadena` RB`.

La etiqueta POS para 'tracks' es `VBZ`
- El siguiente paso es retroceder una palabra en el corpus ('tracks'). Dado que la etiqueta POS más probable para "upward" es "RB", que se identifica de forma única mediante el ID de número entero 28, vaya a la matriz "best_paths" en la columna 2, fila 28. El valor almacenado en "best_paths", columna 2, fila 28 indica el ID único de la etiqueta POS de la palabra anterior. En este caso, el valor almacenado aquí es 40, que es el ID único para la etiqueta POS `VBZ` (verbo, tercera persona del singular presente).
- Entonces, la palabra anterior en el índice 1 del corpus ('tracks'), probablemente tenga la etiqueta POS con ID único 40, que es `VBZ`.
- En la matriz `z`, almacene el valor 40 en la posición 1, y para la matriz` pred`, almacene la cadena `VBZ` para indicar que la palabra 'tracks' probablemente tenga la etiqueta POS` VBZ`.

La etiqueta POS para "loss" es "NN"
- En "best_paths" en la columna 1, el ID único almacenado en la fila 40 es 20. 20 es el ID único para la etiqueta POS "NN".
- En la matriz `z` en la posición 0, almacene 20. En la matriz` pred` en la posición 0, almacene `NN`.


<img src = "Figures/Backwards5.PNG"/>

Implementar el algoritmo `viterbi_backward`, que devuelve una lista de etiquetas POS predichas para cada palabra del corpus.

- Tenga en cuenta que la numeración de las posiciones del índice comienza en 0 y no en 1.
- `m` es el número de palabras del corpus.
     - Entonces, la indexación en el corpus va de `0` a` m - 1`.
     - Además, las columnas en `best_probs` y` best_paths` están indexadas de `0` a` m - 1`
     
**En el paso 1:**
Recorra todas las filas (etiquetas POS) en la última entrada de `best_probs` y busque la fila (etiqueta POS) con el valor máximo.
Convierta el ID entero único en una etiqueta (una representación de cadena) usando el diccionario `estados`.

Refiriéndose al corpus de tres palabras descrito anteriormente:
- `z [2] = 28`: Para la palabra 'hacia arriba' en la posición 2 en el corpus, el ID de la etiqueta POS es 28. Almacene 28 en` z` en la posición 2.
- indica (28) es 'RB': El ID de etiqueta de POS 28 se refiere a la etiqueta de POS 'RB'.
- `pred [2] = 'RB'`: En la matriz` pred`, almacena la etiqueta POS para la palabra' upward '.

**En el paso 2:**
- Comenzando en la última columna de best_paths, use `best_probs` para encontrar la etiqueta POS más probable para la última palabra en el corpus.
- Luego use `best_paths` para encontrar la etiqueta POS más probable para la palabra anterior.
- Actualice la etiqueta POS para cada palabra en `z` y en` preds`.

Refiriéndose al ejemplo de tres palabras de arriba, lea best_paths en la columna 2 y complete z en la posición 1.
`z [1] = best_paths [z [2], 2]`

La pequeña prueba que sigue imprime las últimas palabras del corpus y sus estados para ayudar en la depuración.



# Predecir sobre un conjunto de datos

Calcular la precisión de las predicciones comparándola con las etiquetas 'y'.
- `pred` es una lista de etiquetas POS predichas correspondientes a las palabras del` test_corpus`.



Implementar una función para calcular la precisión de las predicciones de etiquetas POS del algoritmo de Viterbi.
- Para dividir y en la palabra y su etiqueta, puede usar `y.split ()`.
